In [31]:
"""국토교통부 공시지가 데이터
- 링크: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15059127
"""
from dotenv import load_dotenv
import os
import math
import requests
import pandas as pd
import io
import xmltodict
import json
import tqdm

# import xml.etree.ElementTree as et


# load .env
load_dotenv()
getRTMSDataSvcAptTradeDev_en = os.environ.get("getRTMSDataSvcAptTradeDev_en")
getRTMSDataSvcAptTradeDev_de = os.environ.get("getRTMSDataSvcAptTradeDev_de")

In [32]:
start_date = pd.to_datetime('2010-01-01')
end_date = pd.to_datetime('2023-09-30')
dates = pd.date_range(start_date, end_date, freq="Y")

In [33]:
YEAR_LIST = [date.strftime("%Y") for date in dates]

In [45]:
dict_list = []

for year in tqdm.tqdm(YEAR_LIST):

    SIGUNGU_CODE = "11470"
    B_CODE = "10200"
    url = "http://apis.data.go.kr/1611000/nsdi/IndvdLandPriceService/attr/getIndvdLandPriceAttr"
    params = {
        "serviceKey": getRTMSDataSvcAptTradeDev_de,
        "pnu": SIGUNGU_CODE + B_CODE,  # 필지 고유번호
        "stdrYear": year,
        "format": "xml",  # 응답결과 형식(xml 또는 json)
        "numOfRows": 10,  # 검색 건수
        "pageNo": 1,  # 페이지 번호
    }

    response = requests.get(url, params=params)
    rows_count = xmltodict.parse(response.content)["response"]["totalCount"]
    # print(rows_count)
    GET_ROWS_BY_ITER = 10000
    dict_content = xmltodict.parse(response.content)["response"]
    iter_range = math.ceil(int(rows_count) / GET_ROWS_BY_ITER)

    for i in tqdm.tqdm(range(1, iter_range + 1)):
        params = {
            "serviceKey": getRTMSDataSvcAptTradeDev_de,
            "pnu": SIGUNGU_CODE + B_CODE,  # 필지 고유번호
            "stdrYear": year,
            "format": "xml",  # 응답결과 형식(xml 또는 json)
            "numOfRows": GET_ROWS_BY_ITER,  # 검색 건수
            "pageNo": i,  # 페이지 번호
        }

        response = requests.get(url, params=params)
        try:
            dict_content = xmltodict.parse(response.content)["response"]["fields"]["field"]
            dict_list.extend(dict_content)
        except Exception as e:
            print(f"Error message {e}, Iter {i}")

100%|██████████| 13/13 [00:35<00:00,  2.76s/it]


In [46]:
len(dict_list)

84660

In [47]:
df = pd.DataFrame(dict_list)
df.to_csv("data/서울특별시_양천구_공시지가_속성.csv",
          encoding="utf8",
          index=False
)